## Spring Boot Starters
Starters contain a lot of the dependencies that we need to get a project up and running quickly and with a consistent, supported set of managed transitive dependencies. All official starters follow a similar naming pattern; `spring-boot-starter-*`, where `*` is a particular type of application.

Some major starters:

| Starter       | Description                                                                |
|---------------|--------------------------------------------------------------------------- |
| spring-boot-starter | Core starter, including auto-configuration support, logging and YAML |
| spring-boot-starter-batch | Starter for Spring Batch applications                          |
| spring-boot-starter-cache | Starter ffor Spring cache support                              |
| spring-boot-starter-jdbc | Starter for using JDBC with the HikariCP connection pool        |
| spring-boot-starter-data-jdbc | Starter for using Spring Data JDBC                         |
| spring-boot-starter-data-jpa | Starter for using Spring Data JPA with Hibernate            |
| spring-boot-starter-json | Starter for reading and writing json                            |
| spring-boot-starter-test | Starter for testing Spring Boot applications with JUnit Jupiter, Hamcrest and Mockito |
| spring-boot-starter-web  | Starter for building web applications using Spring MVC. Uses Tomcat as the default embedded container |
| spring-boot-starter-actuator | Starter for using Spring Boot’s Actuator                    |
| spring-boot-starter-logging | Starter for logging using Logback.                           |

## Starter in Details
### spring-boot-starter
Includes the following dependencies (using Spring Boot version 2.4.3):

```
org.springframework.boot:spring-boot:2.4.3
        org.springframework:spring-core:5.3.4
        org.springframework:spring-context:5.3.4
                org.springframework:spring-aop:5.3.4
                org.springframework:spring-bean:5.3.4
                ...
org.springframework.boot:spring-boot-autoconfigure:2.4.3

org.springframework.boot:spring-boot-starter-logging:2.4.3

org.springframework:spring-core:5.3.4

org.yaml:snakeyaml:1.27

jakarta.annotation:jakarta.annotation-api:1.3.5
```

### spring-boot-starter-cache
Includes the following dependencies (using Spring Boot version 2.4.3):

```
org.springframework.boot:spring-boot-starter:2.4.3

org.springframework:spring-context-support:5.3.4
```

### spring-boot-starter-jdbc
Includes the following dependencies (using Spring Boot version 2.4.3):

```
org.springframework.boot:spring-boot-starter:2.4.3

org.springframework:spring-jdbc:5.3.4

com.zaxxer:HikariCP:3.4.5
```

### spring-boot-starter-web
Includes the following dependencies (using Spring Boot version 2.4.3):

```
org.springframework.boot:spring-boot-starter:2.4.3
org.springframework.boot:spring-boot-starter-json:2.4.3
org.springframework.boot:spring-boot-starter-tomcat:2.4.3

org.springframework:spring-web:5.3.4
org.springframework:spring-webmvc:5.3.4
```

## AutoConfiguration
The `@EnableAutoConfiguration` annotation is defined as:

In [ ]:
@Target(ElementType.TYPE)
@Retention(RetentionPolicy.RUNTIME)
@Documented
@Inherited
@AutoConfigurationPackage
@Import(AutoConfigurationImportSelector.class)
public @interface EnableAutoConfiguration {
    // fields
}

Contrary to the expectation, `AutoConfigurationImportSelector` class is not a configuration class. If we look at the documentation for `@Import` we can see that:
> `@Import` Allows for importing `@Configuration` classes, `ImportSelector` and `ImportBeanDefinitionRegistrar` implementations, as well as regular component classes.

`AutoConfigurationImportSelector` implements `DeferredImportSelector` which in turn imports `ImportSelector`. The implementation for `ImportSelector` provides a way to programmatically select which configuration classes to import. For example, the below code snippet shows how different configuration classes are selected based on value of an environment variable:

In [ ]:
public class MyImportSelector implements ImportSelector {
    @Override
    public String[] selectImports (AnnotationMetadata importingClassMetadata) {
      String prop = System.env("environment");
      if ("test".equals(prop)) {
          return new String[]{TestConfig.class.getName()};
      } else {
          return new String[]{ProdConfig.class.getName()};
      }
    }
}

@Configuration
class TestConfig{
    // Beans
}

@Configuration
class ProdConfig{
    // Beans
}

@Configuration
@Import(MyImportSelector.class)
class AppConfig{
}

`DeferredImportSelector` runs after all configuration classes have been processed. `AutoConfigurationImportSelector` gets the list of all configuration classes from `spring.factories` file present in `spring-boot-autoconfigure.jar`. All configuration classes (over 150) are listed here. From this list some configuration classes are removed as per value passed to `exclude` or `excludeName` attribute of `@SpringBootApplication` annotation.

Sample `spring.factories`:

```
org.springframework.boot.autoconfigure.EnableAutoConfiguration=\
org.springframework.boot.autoconfigure.admin.SpringApplicationAdminJmxAutoConfiguration,\
org.springframework.boot.autoconfigure.aop.AopAutoConfiguration,\
org.springframework.boot.autoconfigure.amqp.RabbitAutoConfiguration
```

### Conditional
Spring Boot auto-configuration classes heavily make use of `@Conditional` Spring annotation. Spring infact provides several variations of conditionals such as:
- `@ConditionalOnProperty` : enables bean registration only if an environment property is present and has a specific value. By default, the specified property must be defined and not equal to false.

In [ ]:
@Bean(name = "smsNotification")
// Bean created when there is a property notifocation.service=sms
// we can skip havingValue, that would mean just presence of property is enough
@ConditionalOnProperty(prefix = "notification", name = "service", havingValue = "sms")
public NotificationSender getSmsNotificationSender() {
    return new SmsNotification();
}

- `@ConditionalOnClass`, `@ConditionalOnMissingClass` : allows configuration to be skipped based on the presence or absence of specific classes.

In [ ]:
@Configuration
@ConditionalOnClass(HikariDataSource.class)
public class DatasourceConfiguration {
    @Bean
    public DataSource getDataSource(DataSourceProperties properties) {
      HikariDataSource dataSource = createDataSource(properties, HikariDataSource.class);
      if (StringUtils.hasText(properties.getName())) {
          dataSource.setPoolName(properties.getName());
      }
      return dataSource;
    }
}

- `@ConditionalOnBean` , `@ConditionalOnMissingBean` : allow configurations to be skipped based on the presence or absence of specific beans. We can use the value attribute to specify beans by type, or name to specify beans by name. When placed on a `@Bean` method, the bean class defaults to the return type of the factory method.

Some other variants are `@ConditionalOnNotWebApplication`, `@ConditionalOnExpression` , `@ConditionalOnProperty`, `@ConditionalOnResource`, `@ConditionalOnJava`, etc.

### Custom AutoConfiguration
This section is intended for library authors who would want to develop their own auto-configuration. Auto-configuration classes can be bundled in external jars and still be picked-up by Spring Boot.

Starter dependencies for example provide both auto-configuration code as well as the typical libraries that we would use with it. In case, we just want to include the auto-configurations we need to mark the libraries as optional dependency in Maven.

Spring Boot checks for the presence of a `META-INF/spring.factories` (put this in resources folder) file within the published jar. This file is essentially a properties file. The auto-configuration class should be listed in the file as:

```
org.springframework.boot.autoconfigure.EnableAutoConfiguration=\
com.example.autoconfigure.LibXAutoConfiguration,\
com.example.autoconfigure.LibYAutoConfiguration
```

A typical configuration class would then look like:

In [ ]:
@Configuration
// Some conditions
public class MyAutoConfiguration {

    // Auto-configured beans
    @Configuration
    @ConditionalOnClass(EmbeddedAcmeService.class)
    static class EmbeddedConfiguration {
        @Bean@ConditionalOnMissingBeanpublic 
        EmbeddedAcmeService embeddedAcmeService() {
            // ...
        }
    }
}

## Bootstrapping Spring Boot
A typical Spring Boot application bootstraps as:

In [ ]:
@SpringBootApplication
public class Application{
    public static void main(String[] args){
        SpringApplication.run(Application.class);
    }
}

The `@SpringBootApplication` annotation is a combination of the following:

In [ ]:
@Target(ElementType.TYPE)
@Retention(RetentionPolicy.RUNTIME)
@Documented
@Inherited
@SpringBootConfiguration
@EnableAutoConfiguration
@ComponentScan(excludeFilters = { @Filter(type = FilterType.CUSTOM, classes = TypeExcludeFilter.class),
@Filter(type = FilterType.CUSTOM, classes = AutoConfigurationExcludeFilter.class) })
public @interface SpringBootApplication {
    // fields
}

The `@EnableAutoConfiguration` annotation is what enables all the magic.

### Startup Sequence
One of the most important event during startup is creation of `ApplicationContext`. For this, first Spring deduces the application type using classes present in the classpath. For example, if the classpath has `org.springframework.web.reactive.DispatcherHandler` (and Spring MVC & Jersey servlet absent) then application type is Reactive . If no servlet class is present, then application type is None. Else it is Servlet.

For application type:
- web: `AnnotationConfigServletWebServerApplicationContext`
- reactive: `AnnotationConfigReactiveWebServerApplicationContext`
- none: `AnnotationConfigApplicationContext`

### Application Exit
Each SpringApplication registers a shutdown hook with the JVM to ensure that the `ApplicationContext` closes gracefully on exit. This can be done by annotating a method in class annotated with `@SpringBootApplication` with `@PreDestroy` annotation.

We can also add an `ExitCodeGenerator` bean to return custom exit code:

In [ ]:
@Bean
public ExitCodeGenerator exitCodeGenerator() {
    return () -> 42;
}

### Tomcat Server
Spring boot starter web contains Spring boot starter tomcat. This starter contains embedded Tomcat. Spring boot auto-configuration `ServletWebServerFactoryConfiguration` contains the following bean definition (conditional annotations not shown).  
<img width=700 src="https://thepracticaldeveloper.com/images/posts/uploads/2020/04/auto-configuration-1024.webp" >  
The embedded Tomcat server can be configured using various property values listed [here](https://www.amitph.com/spring-boot-configure-embedded-tomcat/).

## Runners
When the `ApplicationContext` is up and running, Spring Boot will run all the registered `CommandLineRunner` and `ApplicationRunner` beans (in order specified using `Ordered` interface or `@Order` annotation).

Both the variants can run arbitrary code, the difference between `ApplicationRunner` and `CommandLineRunner` is that, on `ApplicationRunner`, instead of raw string arguments passed to the run method, we have an instance of `ApplicationArguments` so that we can access the bootstrap arguments passed when initializing the application.

In [ ]:
@Bean
public ApplicationRunner getApplicationRunner() {
    return new ApplicationRunner() {
        @Override
        public void run(ApplicationArguments args) throws Exception {

        }
    };
}

@Bean
public CommandLineRunner getCommandLineRunner() {
    return new CommandLineRunner(){
      @Override
      public void run(String... args) throws Exception {

      }
    };
}

## Environment
Spring Boot application's `Environment` contain multiple additional `PropertySource`s (note that normal Spring application has java system variables and environment variables only). Spring Boot uses a very particular `PropertySource` order that is designed to allow sensible overriding of values. Later property sources can override the values defined in earlier ones.

1. Default properties (specified by setting `SpringApplication.setDefaultProperties`).
2. `@PropertySource` annotations on `@Configuration` classes.
3. Config data (such as `application.properties` files). These are evaluated in following order:
    i. Application properties packaged inside our jar (`application.properties` and YAML variants).
    ii. Profile-specific application properties packaged inside our jar (`application-{profile}.properties` and YAML variants).
    iii. Application properties outside of our packaged jar (`application.properties` and YAML variants).
    iv. Profile-specific application properties outside of our packaged jar (`application-{profile}.properties` and YAML variants).
4. `RandomValuePropertySource`
5. OS environment variables.
6. Java System properties (`System.getProperties()`).
7. JNDI attributes from `java:comp/env`.
8. `ServletContext` init parameters.
9. `ServletConfig` init parameters.
10. Properties from `SPRING_APPLICATION_JSON` (inline JSON embedded in an environment variable or system property).
11. Command line arguments.
12. `properties` attribute on tests. Available on `@SpringBootTest`
13. `@DynamicPropertySource` annotations in tests.
14. `@TestPropertySource` annotations on tests.

### Command Line Properties
Spring boot converts any command line option arguments (that is, arguments starting with `--`, such as `--server.port=9000`) to a property and adds them to the Spring `Environment`.

### JSON Application Properties
When application starts, any `spring.application.json` or `SPRING_APPLICATION_JSON` properties will be parsed and added to the `Environment`.

```bash
$ SPRING_APPLICATION_JSON='{"my":{"name":"test"}}' 
$ java -jar myapp.jar
$ java -Dspring.application.json='{"my":{"name":"test"}}' -jar myapp.jar
$ java -jar myapp.jar --spring.application.json='{"my":{"name":"test"}}'
```

### Profile Specific Files
Spring Boot will also attempt to load profile-specific files using the naming convention `application-{profile}`. For example, if the application activates a profile named prod and uses YAML files, then both `application.yml` and `application-prod.yml` will be considered.

Profile-specific files always override the non-specific ones. If several profiles are specified, a last-wins strategy applies. We can specify properties for multiple profiles in one `application.yaml` file using the triple dash syntax:

```yaml
spring:
    config:
        activate:
            on-profile: test
name: test-YAML
environment: testing
enabled: false
servers: 
    - www.abc.test.com
    - www.xyz.test.com

---
spring:
    config:
        activate:
            on-profile: prod
name: prod-YAML
environment: production
enabled: true
servers: 
    - www.abc.com
    - www.xyz.com
```

## Type-safe Configuration Properties
Using the `@Value("${property}")` annotation to inject configuration properties can sometimes be cumbersome, which is why Spring Boot provides an alternate way of working with properties. This involves use of `@ConfigurationProperties` and `@EnableConfigurationProperties` annotations and the following rules:
- The prefix defines which external properties will be bound to the fields of the class.
- The classes' property names must match the names of the external properties according to Spring Boot’s *relaxed binding rules*.
- We can define a default values by simply initializing a field with a value.
- The class itself can be package private.
- The classes' fields must have public setters.

Consider the example below:

In [ ]:
@ConfigurationProperties("my.service")  // my.service is prefix of property name
public class MyProperties {

    private boolean enabled;  // my.service.enabled, with a value of false by default. To specify default value, just initialize it
    private InetAddress remoteAddress;  // my.service.remote-address (this is the relaxed binding rule mentioned above),
                                        // with a type that can be coerced from String.
    private final Security security = new Security();

    // getters / setters...

    public static class Security {

        private String username;  // my.service.security.username, with a nested "security" object whose name is determined by the 
                                  // name of the property
        private String password;
        private List<String> roles = new ArrayList<>(Collections.singleton("USER")); // my.service.security.roles, with a collection of 
                                                                                     // String that defaults to USER.

        // getters / setters...
    }
}

In the above example, how did a string property `my.service.remote-address` create an `InetAddress` object? Automatic conversion happens when:
- class provides a constructor that takes a single string as an argument
- class provides a static `valueOf` method that takes a single string as an argument and returns the desired object

In cases where the class is not under our control, we can implement a converter;

In [ ]:
@Component
@ConfigurationPropertiesBinding // this annotation is important
class InetAddressConverter implements Converter<String, InetAddress> {

  @Override
  public InetAddress convert(String source) {
    // create and return a InetAddress object from the String
  }

}

For Spring Boot to create a bean of the `MyProperties` class, we need to add it to the application context in one of several ways:
- First, we can simply let it be part of a component scan by adding the `@Component` annotation:

In [ ]:
@Component
@ConfigurationProperties("my.service")
public class MyProperties {
    // ...
}

- Alternatively, we can use the `@EnableConfigurationProperties` annotation to make our class known to Spring Boot:

In [ ]:
@Configuration
@EnableConfigurationProperties(MyProperties.class)
class MyPropertiesConfiguration {
    // ...
}

In case a wrong value has been passed, for example `my.service.enabled=hello`, this will result in error and Spring Boot application context will not load. To remedy this:

In [ ]:
@ConfigurationProperties(prefix = "myapp.mail", ignoreInvalidFields = true)

Properties that are present in `Environment`, but not in our configuration properties file are ignored. We can also add bean validation annotations to check if properties have been initialised correctly:

In [ ]:
@ConfigurationProperties("my.service")
@Validated
public class MyProperties {

    private boolean enabled;
    @NotNull private InetAddress remoteAddress;  // Will generate BindValidationException if null
    
    // ...
    
    public static class Security {

        @NotEmpty private String username; 
        @NotEmpty private String password;

As well as using `@ConfigurationProperties` to annotate a class, you can also use it on public `@Bean` methods. Doing so can be particularly useful when you want to bind properties to third-party components that are outside of your control:

In [ ]:
@Configuration(proxyBeanMethods = false)
public class ThirdPartyConfiguration {

    @Bean
    @ConfigurationProperties(prefix = "another")
    public AnotherComponent anotherComponent() {
        return new AnotherComponent();
    }

}

## Testing
`spring-boot-starter-test` provides bulk of Spring Boot's test capability. It packages multiple helpful packages like JUnit Jupiter, Hamcrest, JsonPath, AssertJ, Mockito and JSONAssert.

Spring Boot provides an alternative to `@ContextConfiguration` to configure the application context through `@SpringBootTest` annotation. `@SpringBootTest` can also start a webserver:

In [ ]:
@SpringBootTest(
        webEnvironment = SpringBootTest.WebEnvironment.RANDOM_PORT
)
@RunWith(SpringRunner.class)
public class BaseTest {
    @LocalServerPort
    int port; // inject the actual port on which the server is running
}

So how is configuration loaded? Similar to `@ContextConfiguration`, we can pass a list of classes to load context from. Usually, this is not needed though since if no class is specified, Spring Boot would search for class annotated with `@SpringBootApplication` or `@SpringBootConfiguration` and load context (if no inner static `@Configuration` existed).

We can also add to the primary context loaded using `@SpringBootApplication`, by utilising `@TestConfiguration` annotation:

In [ ]:
@RunWith(SpringJUnit4ClassRunner.class)
@SpringBootTest(
        webEnvironment = SpringBootTest.WebEnvironment.NONE
)
public class BaseTest {
    @TestConfiguration
    static class BaseTestConfiguration {
        @Bean
        public String connection() {
            return "jdbc:postgresql://localhost:5432/postgres";
        }
    }

    @Autowired
    private String connection;

    @Test
    public void connectionTest() {
        Assertions.assertThat(connection).contains("postgres");
    }
}

### `@MockBean`
We can use the @MockBean to add mock objects to the Spring application context. The mock will replace any existing bean of the same type in the application context.

If no bean of the same type is defined, a new one will be added. This annotation is useful in integration tests where a particular bean, like an external service, needs to be mocked.